# 🏃 Strava Intelligent Analyst (Smart Commands Demo)

**Presentation Demo**

This notebook demonstrates the **Intelligent Analyst Interface** we built to interact with Strava data. Instead of writing complex code, we use simple, human-readable commands.

---

### 1. Initialization
We start by loading our custom `analyst` module. This module contains all the logic for our Smart Commands. We also enable autoreload so any changes to the backend code are immediately reflected here.

In [1]:
%load_ext autoreload
%autoreload 2
import src.analyst as strava
import warnings
warnings.filterwarnings('ignore')

### 2. Global Overview
First, let's get a high-level picture of the athlete's performance. 
The `show("summary")` command calculates total distance, elevation, and average speed across all historical activities.

In [2]:
strava.show("summary")

Loading activities from data/activities.csv
Loading athlete info from cache...
--- Global Statistics for Siddhanth ---


Metric,Value
Total Activities,82
Total Distance,205.5 km
Total Elevation,434 m
Moving Time,18.1 hrs
Avg Speed,15.6 km/h
Max Speed,41.4 km/h
Min Speed,4.1 km/h


Now, let's see the breakdown of different sports. 
The `show("types")` command generates a Donut Chart visualization to show the distribution of Rides vs. Walks vs. Runs.

In [3]:
strava.show("types")

### 3. Comparison (This Month vs Last Month)
A key question for any athlete is "Am I improving?"
The `compare("month")` command automatically fetches data for the current month and the previous month, comparing them side-by-side across Distance, Elevation, and Activity Count.

In [4]:
strava.compare("month")

### 4. Recent Activities
Let's look at the raw data for our most recent efforts. 
The `show("recent")` command displays a table of the latest activities, which is useful for checking recent consistency or identifying specific activity IDs.

In [5]:
# Show the latest 10 activities
strava.show("recent", limit=10)

,start_date_local,name,type,distance,total_elevation_gain
0,2025-12-12 20:34:15+00:00,Abendspaziergang,Walk,1395.4,0.0
1,2025-12-12 17:39:53+00:00,Spaziergang am Nachmittag,Walk,2852.0,3.6
2,2025-12-09 07:26:15+00:00,Morning Walk,Walk,1410.8,4.2
3,2025-11-29 11:56:17+00:00,Lunch Walk,Walk,644.9,2.2
4,2025-11-29 09:48:46+00:00,Morning Walk,Walk,3094.4,5.4
5,2025-11-26 09:35:49+00:00,Morning Walk,Walk,1206.8,3.6
6,2025-11-19 09:32:54+00:00,Morning Walk,Walk,1296.6,3.2
7,2025-11-16 12:38:36+00:00,Exploring emden,Walk,3068.2,3.1
8,2025-11-15 04:28:50+00:00,Early morning walk,Walk,2821.7,15.4
9,2025-11-12 13:54:52+00:00,Afternoon Walk,Walk,993.7,0.0


### 5. Interactive Mapping

We can drill down into specific activities. 
The `plot("map")` command uses the Google Polyline algorithm to decode the GPS data of an activity and renders an interactive Folium map.

*Arguments:* `index=0` selects the most recent activity.

In [6]:
# Index 0 = Latest Activity
strava.plot("map", index=1)

Displaying map for: Spaziergang am Nachmittag (2025-12-12)


### 6. Smart Filtering (The "Drill Down")

Dashboards often become cluttered with mixed data (e.g., mixing Cycle speeds with Walking speeds). 
The `filter()` command sets a global context for the analyst. Here, we restrict all subsequent analysis to only **Walks**.

In [7]:
# Apply a global filter for 'Walk'
strava.filter(sport="Walk")

✅ Filter applied: Activity Type = 'Walk' (24 activities)


Let's verify the filter worked. 
Running `show("summary")` again should now only reflect statistics for our Walks.

In [8]:
# Verify filter works - Summary should now only show Walk stats
strava.show("summary")

Loading athlete info from cache...
--- Global Statistics for Siddhanth ---


Metric,Value
Total Activities,24
Total Distance,46.4 km
Total Elevation,180 m
Moving Time,9.5 hrs
Avg Speed,4.9 km/h
Max Speed,6.7 km/h
Min Speed,4.1 km/h


### 7. Trend Analysis (Filtered)

Now that we have filtered for 'Walk', we can analyze specific trends.
The `plot("trend")` command creates a time-series chart. We use the `metric="pace"` argument to visualize our walking pace (min/km) over time.

In [9]:
# 'pace' is a smart alias for 'minutes per km'
strava.plot("trend", metric="pace")

### 8. Heatmap

To understand our activity habits, we use a heatmap.
The `plot("heatmap")` command aggregates our data by **Month** vs. **Day of Week**, showing us exactly when we are most active.

In [10]:
strava.plot("heatmap", metric="distance_km")

### 9. Reset

Finally, we can clear our filters to return to the full dataset using `filter(reset=True)`.

In [11]:
strava.filter(reset=True)

✅ Filters cleared. Using all data.


### 7. AI Assistant (Ollama)
We've integrated a local LLM (Ollama) to answer questions about your data.
Ensure `ollama serve` is running in your terminal.

In [12]:
strava.ask("Give me a summary of my performance")

🤖 Asking Ollama (mistral-nemo:latest)...


🏆 Great job! You've completed 82 activities, covering 205.5 km with a total elevation gain of 434m. You've been consistently active, with your last activity being a walk on Dec 12th (1.4km). Keep up the excellent progress! 💪

In [14]:
strava.ask("What all types of exercise I have done?")

🤖 Asking Ollama (mistral-nemo:latest)...


You've participated in two main activities: cycling (Ride) - totaling 58 sessions, and walking (Walk) - with a total of 24 sessions!

In [15]:
strava.ask("Which my fastest and lowest speed while riding and while walking? Give me in a table")

🤖 Asking Ollama (mistral-nemo:latest)...


| Activity Type | Fastest Speed (km/h) | Slowest Speed (km/h) |
|---|---|---|
| Ride | 28.5 | 6.7 |
| Walk | 9.3 | 4.1 |

In [13]:
strava.ask("What should I do to improve based on my recent activity?")

🤖 Asking Ollama (mistral-nemo:latest)...


Great job! Consider mixing up your activities with more rides as they've contributed most of your distance and elevation gain. Also, aim for longer walks, perhaps gradually building up towards the longest single activity you've done so far (5.2 km). Keep it up!

In [17]:
strava.ask("Give me a summary of all the workouts i have done in November 2025, categorise based on Ride and Walk, what aspects can I improve?")

🤖 Asking Ollama (mistral-nemo:latest)...


In November 2025, you completed:
- **Rides**: 14 (longest was 5.2 km)
- **Walks**: 9

Great job! Here are some aspects to focus on for improvement:

1. **Variety**: You did more rides than walks. Try to mix it up more.
2. **Consistency**: There's room to increase the frequency of both ride and walk activities.
3. **Distance & Elevation**: While you've had longer rides, your average distance per activity is around 3.5 km. Aim for a bit further each time and challenge yourself with some inclines!

In [18]:
strava.ask("I am planning to do a 5k run in Feb. How can I get started? Looking at the data do you think I can do it?")

🤖 Asking Ollama (mistral-nemo:latest)...


You're absolutely capable of a 5k run! Here's how to start:

1. **Build your base**: You've already completed distances up to 5.2km, so keep building on that.
2. **Consistency is key**: Aim for at least two runs per week.
3. **Increase distance gradually**: Each week, add a bit more to your longest run. For example:
   - Week 1: 3km
   - Week 2: 4km
   - Week 3: 5km (Your current longest)
   - Keep increasing by 0.5-1km each week until you reach 5km.
4. **Track your progress**: Use a fitness app or watch to monitor distance and pace.

You've got this! I believe in you. 🏆🚶‍♂️